# 0. Name & Z-ID
Matt Borek

z1951125

# Project Gutenberg Dataset

In [1]:
import os
from urllib.request import urlretrieve

def download_text(url):
    local_fname = os.path.basename(url)
    if not os.path.exists(local_fname):
        urlretrieve(url, local_fname)
    return local_fname

In [2]:
fname = download_text('https://www.gutenberg.org/files/17610/17610-8.txt') #(encoding: iso-8859-1)
fname2 = download_text('https://www.gutenberg.org/files/63438/63438-0.txt') #(encoding: utf-8-sig)
fname3 = download_text('https://www.gutenberg.org/files/63439/63439-0.txt') #(encoding: utf-8-sig)

# 1. Read Data